In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/training-set/train_age_dataset.csv')
df.head(2)

In [ ]:
df.drop(['Unnamed: 0','userId'],axis=1,inplace=True)

In [ ]:
df['age_group'].value_counts()

In [ ]:
X=df.drop(['age_group'],axis=1)
y=df['age_group']

In [ ]:
import sklearn
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [ ]:
train_x,test_x,train_y,test_y=train_test_split(X_res,y_res,test_size=.0625,random_state=42)

In [ ]:
xgb_clf = XGBClassifier(tree_method = 'gpu_hist', gpu_id=0, objective = "multi:softmax", num_class=4,n_estimators=1000, max_depth=15,learning_rate=0.005)

In [ ]:
xgb_model = xgb_clf.fit(train_x, train_y)

In [ ]:
xgb_model.score(train_x,train_y)

In [ ]:
xgb_model.score(test_x,test_y)

In [ ]:
from imblearn.under_sampling import NearMiss

In [ ]:
nm = NearMiss()
X_res,y_res=nm.fit_sample(X,y)

In [ ]:
X_res

In [ ]:
from sklearn.preprocessing import StandardScaler
features = ['tier', 'gender', 'following_rate', 'followers_avg_age',
       'following_avg_age', 'max_repetitive_punc',
       'num_of_hashtags_per_action', 'emoji_count_per_action',
       'punctuations_per_action', 'number_of_words_per_action',
       'avgCompletion', 'avgTimeSpent', 'avgDuration', 'avgComments',
       'creations', 'content_views', 'num_of_comments',
       'weekends_trails_watched_per_day', 'weekdays_trails_watched_per_day',
       'slot1_trails_watched_per_day', 'slot2_trails_watched_per_day',
       'slot3_trails_watched_per_day', 'slot4_trails_watched_per_day', 'avgt2']
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
y = df.loc[:,['age_group']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=17)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10', 'f11', 'f12','f13', 'f14', 'f15','f16', 'f17'])
finalDf = pd.concat([principalDf, df[['age_group']]], axis = 1)

In [ ]:
X=finalDf.drop(['age_group'],axis=1)
y=finalDf['age_group']

In [ ]:
train_x,test_x,train_y,test_y=train_test_split(X,y,test_size=.0625,random_state=42)

In [ ]:
xgb_model.score(train_x,train_y)

In [ ]:
xgb_model.score(test_x,test_y)

In [ ]:
df1=pd.read_csv('../input/test-set/test_age_dataset.csv')

In [ ]:
df1.head(2)

In [ ]:
df1.drop(['Unnamed: 0','userId'],axis=1,inplace=True)

In [ ]:
pred1=xgb_model.predict(df1)

In [ ]:
pred1

In [ ]:
column_values = ['prediction'] 
test_df = pd.DataFrame(data = pred1,columns = column_values) 
test_df.head(2)

In [ ]:
test_df.reset_index(drop=True, inplace=True)

In [ ]:
test_df.to_csv('k3.csv', index=False)